Other things to try still

- training
- scoring results
- Building our model/modifying this one to include things like augmentation


# Installs

In [ ]:
!git clone https://github.com/SungFeng-Huang/SSL-pretraining-separation.git

In [ ]:
!git clone https://github.com/asteroid-team/asteroid.git

In [ ]:
pip install -e asteroid/.

In [1]:
pip install musdb

# Imports

In [2]:
import sys
import os
sys.path.append(os.path.abspath('/content/SSL-pretraining-separation'))
from src.models import sepformer_tasnet
from asteroid.data import MUSDB18Dataset
import torch
import musdb
from torch.optim import Adam
from asteroid.engine.system import System
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
import pytorch_lightning as pl

# Dataset

In [3]:
#Downloading dataset
db = musdb.DB(download=True)

Done!


In [4]:
#Converting dataset from stem.mp4 files to wav files to make compatible with asteroid
!musdbconvert "../root/MUSDB18/MUSDB18-7" "../root/MUSDB18/MUSDB18-7-wav"

100% 144/144 [05:10<00:00,  2.16s/it]


In [6]:
#torch dataset objects
train_dataset = MUSDB18Dataset(root='../root/MUSDB18/MUSDB18-7-wav', suffix='.wav', split='train')
test_dataset = MUSDB18Dataset(root='../root/MUSDB18/MUSDB18-7-wav', suffix='.wav', split='test')

94it [00:00, 2797.75it/s]
50it [00:00, 2472.39it/s]


In [7]:
#dataloaders which load batches of our datasets during training and validation
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, drop_last=True, collate_fn= lambda x: make_tensor(x))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, drop_last=True, collate_fn= lambda x: make_tensor(x))

In [8]:
def make_tensor(x):
  x = list(x[0])
  dict_y = x[1]
  x[1] = torch.stack([torch.narrow(dict_y['bass'], 0,0, 1), torch.narrow(dict_y['vocals'], 0, 0, 1), torch.narrow(dict_y['drums'], 0, 0, 1), torch.narrow(dict_y['other'], 0, 0, 1)],dim=0).squeeze()
  x[0] = torch.narrow(x[0], 0,0, 1).unsqueeze(0)
  x[1] = x[1].unsqueeze(0)
  return x

# Model

In [9]:
#Initialize model
model = sepformer_tasnet.SepFormerTasNet(n_src=4,n_filters=2,n_heads=2)

# Optimizer & Loss

In [10]:
optimizer = Adam(model.parameters(), lr=1e-5)
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from='pw_mtx')

# Training

In [ ]:
system = System(model=model, loss_func=loss_func, optimizer=optimizer,
                train_loader=train_loader, val_loader=test_loader)
# Define lightning trainer, and train
trainer = pl.Trainer( log_every_n_steps=5,gpus=1)
trainer.fit(system)

In [1]:
system

[]

In [26]:
print(g[1])

{'vocals': tensor([[[ 0.0580,  0.0469,  0.0109,  ..., -0.0008, -0.0009, -0.0009],
         [ 0.0635,  0.0524,  0.0130,  ...,  0.0004,  0.0004,  0.0004]]]), 'bass': tensor([[[-8.4503e-02, -1.0553e-01, -9.1248e-02,  ...,  3.0518e-05,
           3.0518e-05,  3.0518e-05],
         [-8.8501e-02, -1.1063e-01, -9.6008e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]]), 'drums': tensor([[[-2.9602e-03,  4.4556e-03,  1.3702e-02,  ..., -3.0518e-05,
           9.1553e-05,  0.0000e+00],
         [-1.9897e-02, -1.0651e-02,  9.8877e-03,  ...,  3.0518e-05,
           1.2207e-04,  3.0518e-05]]]), 'other': tensor([[[ 0.0121,  0.0146,  0.0117,  ..., -0.0004, -0.0004, -0.0004],
         [ 0.0048,  0.0058,  0.0049,  ...,  0.0004,  0.0004,  0.0003]]])}


In [ ]:
print(torch.narrow(j, 1, 1, 1).shape)

torch.Size([1, 1, 300032])


In [13]:
g[1]['bass'].shape

torch.Size([1, 2, 300032])

# Encoder

# Masking Network

# Decoder